# 📚 מחברת אינטראקטיבית: עבודה עם מחרוזות ב-Polars

**ברוכים הבאים!** מחברת זו מיועדת לתרגול אינטראקטיבי של עבודה עם מחרוזות.

---

## ⚙️ הגדרות ראשוניות

תחילה, נטען את הספריות הנדרשות ונגדיר את הפרמטרים הבסיסיים.

In [ ]:
# ייבוא הספריות
import polars as pl
import os

# הגדרת אורך תצוגה של מחרוזות
os.environ['POLARS_FMT_STR_LEN'] = str(50)

print("✅ הספריות נטענו בהצלחה!")

## 📁 טעינת נתונים

נטען את קובץ הנתונים שלנו (ביקורות מחנות Google Play).

In [ ]:
# טעינת הנתונים
df = pl.read_csv('../data/google_store_reviews.csv')

# הצגת מידע בסיסי
print(f"📊 מספר שורות: {df.height}")
print(f"📊 מספר עמודות: {df.width}")
print("\n5 השורות הראשונות:")
df.head()

---

# 🔍 חלק 1: סינון שורות לפי תנאים

נלמד כיצד לסנן שורות על בסיס תנאים על מחרוזות.

## תרגיל 1.1: סינון לפי תחילת מחרוזת

**משימה:** סנן את כל הביקורות שמתחילות במילה "Very"

In [ ]:
# 👇 כתוב את הקוד שלך כאן
result = (
    df
    .filter(pl.col('content').str.starts_with('Very'))
    .select('content')
    .head()
)
result

## תרגיל 1.2: סינון לפי סיומת

**משימה:** מצא את כל המשתמשים ששם המשפחה שלהם הוא "Smith"

In [ ]:
result = (
    df
    .filter(pl.col('userName').str.ends_with('Smith'))
    .select('userName')
    .head()
)
result

## תרגיל 1.3: חיפוש מילה במחרוזת

**משימה:** מצא ביקורות המכילות את המילה "happy"

In [ ]:
# 👇 כתוב את הקוד שלך
(df.filter(pl.col('content').str.contains('happy', literal=True))
 .select('content')
 .head())


## 💪 אתגר: חיפוש מורכב

**משימה:** מצא ביקורות שמכילות אחת מהמילים: "excellent", "amazing", "wonderful"

In [ ]:
# 👇 נסה לפתור בעצמך
print(df.filter(pl.col('content').str.contains(r'excellent|amazing|wonderful'))
  .select('content'))
print(df.filter(pl.col('content').str.contains_any(['excellent','amazing','wonderful'])).height)


---

# 📅 חלק 2: המרת מחרוזות לתאריכים

נלמד כיצד להמיר מחרוזות לטיפוסי תאריך ושעה.

## תרגיל 2.1: המרה לתאריך

**משימה:** המר את עמודת `at` לטיפוס Date

In [ ]:
print(df.select('at').head())


In [ ]:
result = df.select(
    'at',
    pl.col('at').str.to_date(format='%Y-%m-%d %H:%M:%S').alias('date_only')
).head()
print(result)

## תרגיל 2.2: המרה לשעה

**משימה:** חלץ רק את השעה (Time) מעמודת `at`

In [ ]:
result = df.select('at',pl.col('at').str.to_time(format='%Y-%m-%d %H:%M:%S').alias('time_only')).head()
print(result)

## 💪 אתגר: המרה מלאה

**משימה:** צור שלוש עמודות חדשות: date, time, datetime

In [ ]:
# 👇 נסה לפתור בעצמך
result = (
    df.select(
        'at',
        pl.col('at').str.to_time(format='%Y-%m-%d %H:%M:%S').alias('time_only'),
        pl.col('at').str.to_date(format='%Y-%m-%d %H:%M:%S').alias('date_only'),
        pl.col('at').str.to_datetime(format='%Y-%m-%d %H:%M:%S').alias('datetime_only'))
        .head())
print(result)



---

# ✂️ חלק 3: חילוץ חלקי מחרוזת

נלמד טכניקות שונות לחילוץ חלקים ממחרוזות.

## תרגיל 3.1: חיתוך בסיסי

**משימה:** חלץ את כל התווים החל מהתו ה-4 בשם המשתמש

In [ ]:
result = df.select(
    'userName',
    pl.col('userName').str.slice(3).alias('from_4th_char')
).head()
result

## תרגיל 3.2: חילוץ עם Regex

**משימה:** חלץ את המילה הראשונה מכל ביקורת

In [ ]:
# חילוץ דפוס מורכב: 3 אותיות + רווח + מספר
result = df.select(
    'content',
    pl.col('content')
    .str.extract(r'([A-Za-z]{3}) ([0-9]+)', 0)
    .alias('extract whole matches'),
    pl.col('content')
    .str.extract(r'([A-Za-z]{3}) ([0-9]+)', 1)
    .alias('extract group 1'),
    pl.col('content')
    .str.extract(r'([A-Za-z]{3}) ([0-9]+)', 2)
    .alias('extract group 2')
).head(5)
print(result)

## 💪 אתגר: חילוץ מורכב

**משימה:** חלץ את כל המילים מכל ביקורת לרשימה

In [ ]:
# 👇 נסה לפתור בעצמך



---

# 🧹 חלק 4: ניקוי מחרוזות

נלמד כיצד לנקות ולעצב מחרוזות.

In [ ]:
# יצירת DataFrame לתרגול
df_clean = pl.DataFrame({
    'text': [
        '  Hello World  ',
        'python IS awesome',
        ' DATA science '
    ]
})
df_clean

## תרגיל 4.1: הסרת רווחים והמרה לאותיות קטנות

**משימה:** נקה את הטקסט והמר לאותיות קטנות

In [ ]:
# 👇 כתוב את הקוד שלך



## תרגיל 4.2: החלפת טקסט

**משימה:** החלף את כל ה-spaces ב-underscores

In [ ]:
# 👇 כתוב את הקוד שלך



---

# ✂️ חלק 5: פיצול מחרוזות

נלמד כיצד לפצל מחרוזות לחלקים.

## תרגיל 5.1: פיצול פשוט

**משימה:** פצל ביקורות למילים

In [ ]:
# טען מחדש את הנתונים
df = pl.read_csv('../data/google_store_reviews.csv')

result = df.select(
    'content',
    pl.col('content').str.split(by=' ').alias('words')
).head()
result

## 💪 אתגר: ספירת מילים

**משימה:** צור עמודה חדשה עם מספר המילים בכל ביקורת

In [ ]:
# רמז: השתמש ב-.list.len() אחרי .split()



---

# ➕ חלק 6: שרשור מחרוזות

נלמד כיצד לשרשר מחרוזות מעמודות שונות.

In [ ]:
# יצירת DataFrame לתרגול
df_concat = pl.DataFrame({
    'first_name': ['John', 'Jane', 'Bob'],
    'last_name': ['Doe', 'Smith', 'Johnson']
})
df_concat

## תרגיל 6.1: שרשור שמות

**משימה:** צור עמודת "full_name" ששרשרת את first_name ו-last_name עם רווח

In [ ]:
# 👇 כתוב את הקוד שלך



## 💪 אתגר סופי: עיבוד מלא

**משימה מורכבת:** 
1. טען את נתוני הביקורות
2. נקה רווחים מהביקורות
3. המר לאותיות קטנות
4. סנן רק ביקורות חיוביות (המכילות: good, great, excellent)
5. חלץ את המילה הראשונה
6. ספור כמה ביקורות יש

In [ ]:
# 👇 נסה לפתור את האתגר המלא



---

# 🎉 סיכום

**כל הכבוד!** סיימת את המחברת האינטראקטיבית.

## 📚 מה למדת?

- ✅ סינון שורות לפי תנאים על מחרוזות
- ✅ המרת מחרוזות לתאריכים ושעות
- ✅ חילוץ חלקי מחרוזת עם slice ו-Regex
- ✅ ניקוי וטיפול במחרוזות
- ✅ פיצול מחרוזות לרשימות
- ✅ שרשור מחרוזות

## 🚀 המשך לתרגל!

נסה ליצור תרגילים משלך עם נתונים אחרים.